In [3]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

# Intuitive Example

 Assume, the underlying true function $f$ that dictates the relationship between $x$ and $y$ is:

$ f(x) = \frac{1}{2} x + \sqrt{max(x, 0)}-cosx+2 $


In [ ]:
def f(x):
    return .5 * x + np.sqrt(np.max(x, 0)) - np.cos(x) + 2

and the noise is modeled by a Gaussian with zero mean and standard deviation 1, $ϵ \approx 𝒩(0, 1)$. 

In [ ]:
sigma_epsilon = 1

As a reminder, $y = f(x) + ϵ$. Let's randomly generate 1,000 points from this process:

In [ ]:
N = 1000

#generating x values for this example
x_max = 3
x = 3 * (2 * np.random.rand(N) - 1)

#generating epsilon values for this example
epsilon = sigma_epsilon * np.random.randn(N)

#generating y values for this example
y = f(x) + epsilon

#generating y_test values for this example
y_test = f(3.2) + sigma_epsilon * np.random.randn()

As a result, we get the following plot:

In [ ]:
#plot the graph
plt.figure(figsize=(12, 6))
x_range = np.linspace(-x_max, x_max, 1000)
plt.scatter(x, y)
plt.plot(x_range, f(x_range), 'r', linewidth=3.0)
plt.scatter(x_test, y_test, c='r')
plt.xlabel('x', size=12)
plt.ylabel('y', size=12)
plt.xticks(np.arange(-x_max, x_max + 1))
plt.show()

Blue dots represent $(x, y)$ pairs and red line is the underlying true function $f(x)$. Red dot is the unseen (test) point we want to predict. We see that f follows a non-linear pattern due to the addition of square root and cosine in the function’s definition. For our purposes, these 1,000 points represent the whole underlying population. Here is the code to reproduce this plot.

# Polynomial Regression

Let's now model the problem with polynomial regressions of varying degrees of complexity. As a reminder, in polynomial regression we try to fit the following non-linear relationship between $x$ and $y$:

$\hat{f}(x) = w_0 + w_1x+w_2x^2+ . \ . \ . + w_dx^d$

In other words, we try to approximate $y$ with $\hat{f}(x)$

In [ ]:
def f_hat(x, w):
    d = len(w) - 1
    return np.sum(w * np.power(x, np.expand_dims(np.arange(d, -1, -1), 1)).T, 1)


Now, let’s assume we could only use 20 points (out of the 1,000) to train our polynomial regression model and we consider four different regression models, one with degree d=1 (simple line), one with d=2, d=3 and d=5.

In [ ]:
# defining training points to be 20
n = int(.02 * N)

#defining other stuff
x_test = 3.2
x_range = np.linspace(-x_max, x_max, 1000)
colors = np.array(['tab:green', 'tab:purple', 'tab:cyan', 'tab:orange'])

#defining degrees
d_arr = [1, 2, 3, 5]

If we randomly sample 20 points from the underlying population and we repeat this experiment 6 times, this is a possible outcome we get.

In [ ]:
#plot the graphs
cnt = 1
fig, axs = plt.subplots(2, 3, sharey=True, figsize=(15, 9))
for i in range(2):
    for j in range(3):
        idx = np.random.permutation(N)[:n]
        x_train, y_train = x[idx], y[idx]
        
        w = []
        for d in d_arr:
            w.append(np.polyfit(x_train, y_train, d))
                
        axs[i, j].scatter(x_train, y_train)
        axs[i, j].plot(x_range, f(x_range), 'r', linewidth=3.0)
        for k in range(len(w)):
            axs[i, j].plot(x_range, f_hat(x_range, w[k]), colors[k], linewidth=3.0)
            
        axs[i, j].scatter(x_test, y_test, c='r')
        for k in range(len(w)):
            axs[i, j].scatter(x_test, f_hat(x_test, w[k]), c=colors[k])
                
        axs[i, j].set_xlabel('x', size=12)
        axs[i, j].set_ylabel('y', size=12)
        axs[i, j].legend([r'$f$', r'$\hat{f}$ (d = 1)', r'$\hat{f}$ (d = 2)', 
                          r'$\hat{f}$ (d = 3)', r'$\hat{f}$ (d = 5)'], fontsize=12)
        axs[i, j].title.set_text('experiment {}'.format(cnt))
        cnt += 1
plt.tight_layout()
plt.show()

Blue dots represent the 20 training data points for a specific realization (experiment). The red line is the underlying (unknown to us) true function $f$ and the other lines represent the fitting of the four different models to different realizations of training data. The green, purple, cyan, orange dots represent the prediction $\hat{f}(x)$ of test (unseen) point x under each model. As we see, there’s less variation in lines with smaller degrees of complexity. Take for instance $d=1$ (simple line). The slope of the line does not change all that much between different experiments. On other other hand, a more complex model ($d=5$) is much more sensitive to small fluctuations in the training data. See for example the difference in the orange line ($d=5$) between experiment 1 and 6 and how this affects prediction $\hat{f}(x)$. This is the variance problem we mentioned in previous sections. A simplistic model is very robust to changes in training data, but a more complex is not. On other hand, the deviation of $\hat{f}(x)$ from $f(x)$ on average (the bias), is larger for more simplistic models, since our assumptions are not as representative of the underlying true relationship $f$. Here is the code for the above plots.

# Bias-Variance Graph

Let's make some data for this example:

In [6]:
R = 10000
n = int(.02 * N)
n_test = 1000
x_max = 3
d_arr = np.arange(5)

#generating x_test and epsilon values for this example
x_test = x_max + np.random.rand(n_test) - .5
epsilon = sigma_epsilon * np.random.randn(n_test)

#generating y_test values for this example
y_test = f(x_test) + epsilon

#generating train_squared_error values for this example
train_squared_error = np.zeros((len(d_arr), R))

#generating y_hat_test values for this example
y_hat_test = np.zeros((len(d_arr), R, n_test))
for r in range(R):
    n = int(.02 * N)
    idx = np.random.permutation(N)[:n]
    x_train, y_train = x[idx], y[idx]
    for k in range(len(d_arr)):
        d = d_arr[k]
        w = np.polyfit(x_train, y_train, d)
        train_squared_error[k, r] = np.mean((y_train - f_hat(x_train, w)) ** 2)
        y_hat_test[k, r, :] = f_hat(x_test, w)

NameError: name 'x' is not defined

Now let’s consider 1,000 test points and compute the average test MSE (over these points). We also compute the average squared bias (over these 1,000 test points) and average variance.

$ MSE = E \ [(y-\hat{f}(x))^2] $

In [ ]:
# defining MSE error       
test_squared_error = np.mean((y_hat_test - y_test) ** 2, 1)

$ Bias = ( \ E[\hat{f}(x)] - f(x) \ )^2 $

In [ ]:
# defining bias
bias_squared = (np.mean(y_hat_test, 1) - f(x_test)) ** 2

$ Variance = E \ [(\hat{f}(x)-E[\hat{f}(x)])^2] $

In [ ]:
#defining variance
var_y_hat_test = np.var(y_hat_test, 1)

If we do this for five models, from degree d=0 (horizontal line) all the way to degree d=4, we get the following plot.

In [ ]:
#plot the graph
plt.figure(figsize=(12, 8))

plt.plot(d_arr, np.mean(test_squared_error, 1), 'g', linewidth=3.0)
plt.plot(d_arr, np.mean(train_squared_error, 1), 'k', linewidth=3.0)
plt.plot(d_arr, np.mean(bias_squared, 1), 'y--')
plt.plot(d_arr, np.mean(var_y_hat_test, 1), 'b--')
plt.plot(d_arr, (sigma_epsilon ** 2) * np.ones_like(d_arr), 'r--')

plt.xticks(d_arr)
plt.xlabel('d', size=12)
plt.legend(['test error', 'training error', r'bias squared: $(\mathbb{E}[\hat{f}(x)] - f(x))^2$',
            r'$var(\hat{f}(x))$', r'irreducible error: $\sigma_\epsilon^2$'], loc='upper center', fontsize=12)

plt.ylim([0, 12])
plt.show()